## load prerequisites

In [1]:
import csv
import random
import numpy as np
import tensorflow as tf

## Preprocess data : names

In [2]:
data_name = set()
max_len = 0
with open('./woman_name_dataset.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
        else:
            tmp_name = row[1].split()[0]
            data_name.add(row[1].split()[0])
            if len(tmp_name) > max_len:
                max_len = len(tmp_name)
data_name = list(data_name)
print('name total : {}'.format(len(data_name)))
print('maximum name length : {}'.format(max_len))

first name total : 1219
maximum name length : 11


## Preprocess data : Characters

In [3]:
chars = set()
for name in data_name:
    for char in name:
        chars.add(char)
chars = list(np.sort(list(chars)))
print('{} alphabets : '.format(len(chars)), chars)

26 alphabets :  ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


## Define function to convert name to onehot

In [4]:
def name_to_onehot(names, chars, max_len):
    onehot = np.zeros((len(names), max_len, len(chars)+1))
    for idx_1, name in enumerate(names):
        for idx_2 in range(max_len):
            if idx_2 < len(name):
                idx_3 = chars.index(name[idx_2])
                onehot[idx_1, idx_2, idx_3] = 1
            else:
                onehot[idx_1, idx_2, -1] = 1
    return onehot

onehot_ex = name_to_onehot(['jane'], chars, max_len)

## Define dimension and Placeholders

In [5]:
num_data = len(data_name)
seq_len = max_len - 1
dim_data = len(chars) + 1

ph_input_name = tf.placeholder(dtype=tf.float32, shape=[None, seq_len, dim_data])
ph_output_name = tf.placeholder(dtype=tf.float32, shape=[None, seq_len, dim_data])

## Define weight variables

In [6]:
dim_rnn_cell = 128
stddev = 0.02

with tf.variable_scope('weights'):
    W_i = tf.get_variable('W_i', dtype=tf.float32,
                         initializer=tf.random_normal([dim_data, dim_rnn_cell],
                         stddev = stddev))
    b_i = tf.get_variable('b_i', dtype=tf.float32,
                         initializer=tf.random_normal([dim_rnn_cell],
                         stddev = stddev))
    W_o = tf.get_variable('W_o', dtype=tf.float32,
                         initializer=tf.random_normal([dim_rnn_cell, dim_data],
                         stddev = stddev))
    b_o = tf.get_variable('b_o', dtype=tf.float32,
                         initializer=tf.random_normal([dim_data],
                         stddev = stddev))

## Define RNN for training

In [7]:
def name_rnn_train(_x, _seq_len, _dim_data, _dim_rnn_cell):
    _x_split = tf.transpose(_x, [1, 0, 2]) # seq_len, batch, dim_data
    _x_split = tf.reshape(_x_split, [-1, _dim_data])
    
    # Load Variables
    
    # Linear Operation for Input
    
    # Define LSTM Cell && RNN
    with tf.variable_scope('rnn', reuse=tf.AUTO_REUSE):
        _rnn_cell = tf.nn.rnn_cell.BasicLSTMCell(_dim_rnn_cell)
        _output, _state = tf.nn.static_rnn(_rnn_cell, _h_split, dtype=tf.float32)
    
    # Linear Operation for Output
    _total_out = []

## Define result graph

In [8]:
result_name = name_rnn_train(ph_input_name, seq_len, dim_data, dim_rnn_cell)
print('result_shape :', result_name.shape)

result_shape : (?, 10, 27)


## Define Loss function

In [9]:
def name_loss(_gt_name, _result_name, _seq_len):
    total_loss = 0
    for i in range(_seq_len):
        ## Fill here
    return total_loss
rnn_loss = name_loss(ph_output_name, result_name, seq_len)

## Define Optimizer and Get Ready

In [10]:
learning_rate = 1e-3
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(rnn_loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver(var_list=tf.trainable_variables())

print('Now ready to start the session')

Now ready to start the session


## Session Run

In [82]:
max_epoch = 300
batch_size = 64
num_batch = int(num_data/batch_size)
with tf.Session() as sess:
    sess.run(init)
    
    for _epoch in range(max_epoch):
        random.seed(_epoch)
        batch_shuffle = list(range(num_data))
        random.shuffle(batch_shuffle)
        
        total_train_loss = 0
        
        for i in range(num_batch):
            batch_idx = [batch_shuffle[idx] for idx in range(i*batch_size,
                                                            (i+1)*batch_size)]
            batch_names = [name for name in data_name if data_name.index(name) in batch_idx]
            batch_onehots = name_to_onehot(batch_names, chars, max_len)
            
            input_onehot = batch_onehots[:, 0:(max_len-1), :]
            output_onehot = batch_onehots[:, 1:max_len, :]

            train_feed_dict = {ph_input_name: input_onehot, 
                               ph_output_name: output_onehot}
            
            sess.run(optimizer, feed_dict = train_feed_dict)
            curr_loss = sess.run(rnn_loss, feed_dict=train_feed_dict)
            
            total_train_loss += curr_loss/num_batch

        print('epoch : {}, train_loss : {}'.format(_epoch+1, total_train_loss))
        
        model_save_path = saver.save(sess, './RNN_model/model.ckpt', global_step=_epoch+1)
        print('Model saved in file: {}'.format(model_save_path))

epoch : 1, train_loss : 28.778023769980983
Model saved in file: ./RNN_model/model.ckpt-1
epoch : 2, train_loss : 18.685396294844782
Model saved in file: ./RNN_model/model.ckpt-2
epoch : 3, train_loss : 17.664300617418792
Model saved in file: ./RNN_model/model.ckpt-3
epoch : 4, train_loss : 17.139000440898695
Model saved in file: ./RNN_model/model.ckpt-4
epoch : 5, train_loss : 16.783934392427145
Model saved in file: ./RNN_model/model.ckpt-5
epoch : 6, train_loss : 16.44162032478734
Model saved in file: ./RNN_model/model.ckpt-6
epoch : 7, train_loss : 16.113945659838226
Model saved in file: ./RNN_model/model.ckpt-7
epoch : 8, train_loss : 15.82344230852629
Model saved in file: ./RNN_model/model.ckpt-8
epoch : 9, train_loss : 15.531801324141652
Model saved in file: ./RNN_model/model.ckpt-9
epoch : 10, train_loss : 15.230911154496043
Model saved in file: ./RNN_model/model.ckpt-10
epoch : 11, train_loss : 14.961386429636102
Model saved in file: ./RNN_model/model.ckpt-11
epoch : 12, train_l

epoch : 92, train_loss : 7.646725805182207
Model saved in file: ./RNN_model/model.ckpt-92
epoch : 93, train_loss : 7.603978508397153
Model saved in file: ./RNN_model/model.ckpt-93
epoch : 94, train_loss : 7.536382248527126
Model saved in file: ./RNN_model/model.ckpt-94
epoch : 95, train_loss : 7.48079252243042
Model saved in file: ./RNN_model/model.ckpt-95
epoch : 96, train_loss : 7.424272637618215
Model saved in file: ./RNN_model/model.ckpt-96
epoch : 97, train_loss : 7.384883453971462
Model saved in file: ./RNN_model/model.ckpt-97
epoch : 98, train_loss : 7.32725775869269
Model saved in file: ./RNN_model/model.ckpt-98
epoch : 99, train_loss : 7.253611765409771
Model saved in file: ./RNN_model/model.ckpt-99
epoch : 100, train_loss : 7.200433680885716
Model saved in file: ./RNN_model/model.ckpt-100
epoch : 101, train_loss : 7.162365386360571
Model saved in file: ./RNN_model/model.ckpt-101
epoch : 102, train_loss : 7.1150200994391195
Model saved in file: ./RNN_model/model.ckpt-102
epoch

Model saved in file: ./RNN_model/model.ckpt-181
epoch : 182, train_loss : 5.0195335839924065
Model saved in file: ./RNN_model/model.ckpt-182
epoch : 183, train_loss : 5.028711042906108
Model saved in file: ./RNN_model/model.ckpt-183
epoch : 184, train_loss : 5.003819013896741
Model saved in file: ./RNN_model/model.ckpt-184
epoch : 185, train_loss : 4.9933046792682845
Model saved in file: ./RNN_model/model.ckpt-185
epoch : 186, train_loss : 4.977906653755589
Model saved in file: ./RNN_model/model.ckpt-186
epoch : 187, train_loss : 4.977079090319182
Model saved in file: ./RNN_model/model.ckpt-187
epoch : 188, train_loss : 4.979315155430844
Model saved in file: ./RNN_model/model.ckpt-188
epoch : 189, train_loss : 4.963180993732654
Model saved in file: ./RNN_model/model.ckpt-189
epoch : 190, train_loss : 4.954694522054571
Model saved in file: ./RNN_model/model.ckpt-190
epoch : 191, train_loss : 4.94850166220414
Model saved in file: ./RNN_model/model.ckpt-191
epoch : 192, train_loss : 4.928

epoch : 271, train_loss : 4.614603218279386
Model saved in file: ./RNN_model/model.ckpt-271
epoch : 272, train_loss : 4.595812195225766
Model saved in file: ./RNN_model/model.ckpt-272
epoch : 273, train_loss : 4.59453770988866
Model saved in file: ./RNN_model/model.ckpt-273
epoch : 274, train_loss : 4.596465738196122
Model saved in file: ./RNN_model/model.ckpt-274
epoch : 275, train_loss : 4.586086699837133
Model saved in file: ./RNN_model/model.ckpt-275
epoch : 276, train_loss : 4.584219204752069
Model saved in file: ./RNN_model/model.ckpt-276
epoch : 277, train_loss : 4.582099588293777
Model saved in file: ./RNN_model/model.ckpt-277
epoch : 278, train_loss : 4.58248560052169
Model saved in file: ./RNN_model/model.ckpt-278
epoch : 279, train_loss : 4.578577794526753
Model saved in file: ./RNN_model/model.ckpt-279
epoch : 280, train_loss : 4.573246805291426
Model saved in file: ./RNN_model/model.ckpt-280
epoch : 281, train_loss : 4.572923258731239
Model saved in file: ./RNN_model/model

## Define RNN to test

In [1]:
ph_test_input_name = tf.placeholder(dtype=tf.float32, shape=[1, 1, dim_data])
ph_h = tf.placeholder(dtype=tf.float32, shape=[1, dim_rnn_cell])
ph_c = tf.placeholder(dtype=tf.float32, shape=[1, dim_rnn_cell])

def name_rnn_test(_x, _dim_data, _dim_rnn_cell, _prev_h, _prev_c):
    _x_split = tf.transpose(_x, [1, 0, 2]) # seq_len, batch, dim_data
    _x_split = tf.reshape(_x_split, [-1, _dim_data])
    
    # Load Variables
    
    # Linear Operation for Input

    # Define LSTM Cell && RNN including (h, c)
    
    # Linear Operation for Ouput
    _total_out = []    

NameError: name 'tf' is not defined

## Run Test

In [98]:
test_result_name, test_state = name_rnn_test(ph_test_input_name, dim_data, dim_rnn_cell,
                                            ph_h, ph_c)

with tf.Session() as sess:
    sess.run(init)
    
    saver.restore(sess, './RNN_model/model.ckpt-300')
    
    total_name = ''
    prev_char = 'a'
    total_name += prev_char
    prev_state = (np.zeros((1, dim_rnn_cell)), np.zeros((1, dim_rnn_cell)))
    for i in range(seq_len):
        input_onehot = np.zeros((1, 1, dim_data))
        prev_char_idx = chars.index(prev_char)
        input_onehot[:, :, prev_char_idx] = 1
            
        test_feed_dict = {ph_test_input_name: input_onehot,
                          ph_h: prev_state[0], ph_c: prev_state[1]}
        curr_result, curr_state = sess.run([test_result_name, test_state], test_feed_dict)
        if np.argmax(curr_result) == dim_data-1:
            break
        else:
            softmax_result = sess.run(tf.nn.softmax(test_result_name), test_feed_dict)
            softmax_result = np.squeeze(softmax_result)
            softmax_result = softmax_result[:dim_data-1]/sum(softmax_result[:dim_data-1])
            prev_char = np.random.choice(chars, 1, p=softmax_result)
            total_name += prev_char[0]
            prev_state = curr_state        
    print('Result Name :', total_name)

INFO:tensorflow:Restoring parameters from ./RNN_model/model.ckpt-300
Result Name : alyssa
